In [1]:
import torch
import numpy as np, cv2, pandas as pd, glob, time, random
import matplotlib.pyplot as plt
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision.transforms.v2 as v2
from torch.cuda.amp import autocast, GradScaler
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Running on {device}')

Running on cuda


In [2]:
class FacesData(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self._preprocess_dataset()
        self.range = (self.df['age'].min(), self.df['age'].max())
        self.df['age'] = (self.df['age'] - self.range[0]) / (self.range[1] - self.range[0])
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.loc[index]
        img_path = './FairFace/' + self.df.loc[index, 'file']
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
        img = self.transform(image=img)['image']
        age = torch.as_tensor(row.age, dtype=torch.float32)
        gender = torch.as_tensor(row.gender, dtype=torch.float32)
        return img, age, gender

    def _age_from_range(self, age_range):
        a, b = map(int, age_range.split('-'))
        return random.randint(a, b)

    def _preprocess_dataset(self):
        self.df['age'] = self.df['age'].replace('more than 70', '70-79')
        self.df['age'] = self.df['age'].apply(self._age_from_range).astype(float)
        self.df['gender'] = self.df['gender'].map({'Male': 1, 'Female': 0}).astype(float)
    
    def load_img(self, index):
        img_path = './FairFace/' + self.df.loc[index, 'file']
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)
        return img
    
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=0, p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05, p=0.5),
    A.ToGray(p=0.05),
    A.CoarseDropout(
        max_holes=2,
        max_height=int(224*0.1),
        max_width=int(224*0.1),
        min_holes=1,
        min_height=int(224*0.02),
        min_width=int(224*0.02),
        fill_value=0,
        p=0.2
    ),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])


val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

train_df = pd.read_csv('./FairFace/train_labels.csv')
val_df = pd.read_csv('./FairFace/val_labels.csv')

train_dataset = FacesData(train_df, train_transform)
val_dataset = FacesData(val_df, val_transform)

BATCH_SIZE = 128
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

c:\Users\ramom\Desktop\theRealProject\.venv\Lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\ramom\AppData\Local\Temp\ipykernel_12820\4175202589.py:44: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(


In [3]:
class AgeGenderClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.intermediate = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 64),
            nn.ReLU(),
        )

        self.age_classifier = nn.Sequential(
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

        self.gender_classifier = nn.Sequential(
            nn.Linear(64, 1)
        )
    
    def forward(self, x):
        x = self.intermediate(x)
        age = self.age_classifier(x)
        gender = self.gender_classifier(x)
        return age, gender

def get_model():
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    model.fc = AgeGenderClassifier()

    for name, param in model.named_parameters():
        if "layer3" in name or "layer4" in name or "fc" in name:
            param.requires_grad = True
        else:
            param.requires_grad = False

    model.avgpool = nn.AdaptiveAvgPool2d((1,1))
    criterion1 = nn.L1Loss()
    criterion2 = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    scaler = GradScaler()

    return model.to(device), criterion1, criterion2, optimizer, scheduler, scaler

def process_batch(images, ages, genders, model, c1, c2, opt, scaler, train=True):
    model.train(train)
    with torch.set_grad_enabled(train):
        with autocast():
            age_pred, gender_pred = model(images)
            loss1 = c1(age_pred, ages)
            loss2 = c2(gender_pred, genders)
            loss = loss1 + loss2

        if train:
            opt.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()
        
    return age_pred, gender_pred, loss1, loss2

def calculcate_accuracy(gender_pred, genders):
    gender_pred = torch.round(torch.sigmoid(gender_pred))
    is_correct = gender_pred == genders
    return sum(is_correct).item(), len(is_correct)


In [4]:
model, c1, c2, opt, scheduler, scaler = get_model()
train_accs, train_losses = [], []
val_accs, val_losses = [], []

for epoch in range(30):
    total_loss = 0
    n_correct = 0
    n_samples = 0
    for i, (images, ages, genders) in enumerate(train_loader):
        images = images.to(device)
        ages = ages.to(device).unsqueeze(1)
        genders = genders.to(device).unsqueeze(1)
        age_pred, gender_pred, loss1, loss2 = process_batch(images, ages, genders, model, c1, c2, opt, scaler, True)
        total_loss += (loss1 + loss2)
        accuracy = calculcate_accuracy(gender_pred, genders)
        n_correct += accuracy[0]
        n_samples += accuracy[1]
        # if i % 100 == 0:
        #     print(f'Batch #{i}')
    train_accs.append(n_correct / n_samples)
    train_losses.append(total_loss / len(train_loader))
    print(f'Epoch {epoch+1}, Train Acc: {(train_accs[-1]):.2f}, Train MAE: {(train_losses[-1]):.2f}', end='')
    
    total_loss = 0
    n_correct = 0
    n_samples = 0
    for i, (images, ages, genders) in enumerate(val_loader):
        images = images.to(device)
        ages = ages.to(device).unsqueeze(1)
        genders = genders.to(device).unsqueeze(1)
        age_pred, gender_pred, loss1, loss2 = process_batch(images, ages, genders, model, c1, c2, opt, scaler, False)
        total_loss += (loss1 + loss2)
        accuracy = calculcate_accuracy(gender_pred, genders)
        n_correct += accuracy[0]
        n_samples += accuracy[1]
        # if i % 100 == 0:
        #     print(f'Batch #{i}')
    val_accs.append(n_correct / n_samples)
    val_losses.append(total_loss / len(val_loader))
    print(f', Val Acc: {(val_accs[-1]):.2f}, Val MAE: {(val_losses[-1]):.2f}')
    scheduler.step(val_losses[-1])

C:\Users\ramom\AppData\Local\Temp\ipykernel_12820\1596619897.py:42: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
C:\Users\ramom\AppData\Local\Temp\ipykernel_12820\1596619897.py:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
len(train_loader)

1356

In [ ]:
train_df['gender'].value_counts()

gender
1.0    45986
0.0    40758
Name: count, dtype: int64